Assignment 5

Agents are an emerging field thats use reflection, tools, planning, and multi agent collaboration


In this assignment, we will build a research agent. We will use serverless LLM endpoints. To get started, you create an account with Together AI or Anthropic. They should provide you with a few dollars worth of credits that should be enough to complete the assignment. You are free to choose any other provider such as OpenAI, Mistral, Fireworks, OctoAI, or Groq. I encourage you to play around with different models to get a feel for how they work. For this assignment, the API usage cost should be around a couple dollars. Depending on the model you choose and how many attempts you use, it may be a couple cents. For OpenAI, Anthropic, and Mistral, double check what model you are using. The flagship models are significantly more expensive than the smaller models (pricing between models varies by 50x). For the purposes of this assignment, it is sufficient to use the smallest/cheapest models.
TogetherAI, Fireworks, OctoAI, and Groq run open source models. For these, it’s better to run the mid-large tier models. Mixtral is a good place to start. It’s good to play around with different models.
Many providers are able to use OpenAI's client library, but some do not (like Anthropic). Use whatever makes sense.
You can run this on Colab with a CPU, or locally and submit the Jupyter notebook as your submission. Since we are using third party providers for the LLMs, we will not load the model locally. If you run on Colab, take special care to not leak your API key. Here’s an example of how to properly use secrets in Colab.

Research Agent Build an LLM-based research agent that can take a research topic, find relevant information, and generate a short summary (~1 paragraph) on the given topic.
Tools to Implement (20 points, 4 points each):
Topic Breakdown Tool: Create a tool that takes a broad research topic and breaks it down into smaller, more focused subtopics or subqueries. You can use an LLM to generate these subtopics based on the main topic.
Query Expansion Tool: Develop a tool to expand the subqueries generated by the Topic Breakdown Tool. The tool should generate related keywords, synonyms, and phrases to enhance the search results.
Search Tool: Create a wrapper around the You API or Brave Search API. Please note that the free tier is 1000 queries/month. Consider creating a mock while developing, and switch to actually call the You API once the agent is more stable. Additionally, consider caching the search results.
Critique Tool: Create a tool that critiques the summary, and offers suggestions of how to improve and potentially other relevant topics to search for.
Summarizer Tool (optional): Create a tool that takes some input and summarizes its content using an LLM.

Workflow (30 points) Implement an agent workflow that uses all of these tools. In the agent workflow, the agent should be provided with all the tools and it should decide which tool to use. For the individual tool implementations, if you use a call to an LLM you do not need to provide any tools.
Sample Agent Workflow:
The agent receives a research topic from the user.
It uses the Topic Breakdown Tool to generate subtopics or subqueries.
The Query Expansion Tool expands the subqueries with related keywords and phrases.
The Search Tool uses the expanded queries and subqueries to gather relevant information from various sources.
The agent generates the summary incorporating the search results. (optional)
The agent critiques the summary, and improves the results. (optional)
The agent presents the final summary to the user.
The sample workflow is the minimum implementation requirement. Feel free to add more tools, add loops in the workflow, etc.

In [ ]:
pip install --upgrade together

In [ ]:
pip install openai

In [ ]:
!pip show openai

Name: openai
Version: 1.23.2
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [ ]:
from openai import OpenAI
import os

In [ ]:
from google.colab import userdata
my_secret_key = userdata.get('Together_api_token')
my_secret_key_2=userdata.get('You_api_key')

In [ ]:
import together
client = together.Client(api_key=my_secret_key,base_url='https://api.together.xyz/v1',)

In [ ]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [ ]:
def topic_breakdown_tool(research_topic):
      chat_completion = client.chat.completions.create(
        messages=[
          {
            "role": "system",
            "content": "You are an AI research assistant.",
          },
          {
            "role": "user",
            "content": f"Generate 4 subQUERIES based on the research topic: {research_topic}.",
          }
        ],
        model=model_name,
        temperature=0.9,
        max_tokens=256,
        top_k=40
      )
      subqueries=chat_completion.choices[0].message.content

      return subqueries



In [ ]:
def query_expansion_tool(subqueries):
    conversation = [
        {"role": "system", "content": "You are an AI research assistant."},
        {"role": "user", "content": f"Generate 4 keywords, synonyms, and phrases for each of the subqueries:{subqueries} and should follow the following format: {{'Subquery':Subquery,'Keywords':keywords,'Synonyms':Synonyms,'Phrases':phrases}}"}
    ]
    # Request chat completions for generating keywords based on the conversation
    chat_completion = client.chat.completions.create(
        messages=conversation,
        model=model_name,
        temperature=0.9,
        max_tokens=512,
        top_k=40
    )

    # Extract keywords from the completion
    expanded_queries = chat_completion.choices[0].message.content
    return expanded_queries


In [ ]:
import requests
from cachetools import TTLCache
from datetime import datetime, timedelta

# Initialize a cache with a time-to-live (TTL) of 3600 seconds (1 hour)


def search_tool(query):
    headers = {"X-API-Key": my_secret_key_2}
    params = {"query": query}
    response= requests.get(
        f"https://api.ydc-index.io/search?query={query}",
        params=params,
        headers=headers,
    ).json()
    return response




In [ ]:
def summary_tool(search_result):
    messages = [
        {"role": "system", "content": "You are an AI research assistant."},
        {"role": "user", "content": f"Provide Summary for the following search results generated:\n\n{search_result}."}
    ]
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        max_tokens=500,
        stop=None,
        temperature=0.8,
    )
    summary = response.choices[0].message.content
    return summary

In [ ]:
def critique_tool(summary):
    messages = [
        {"role": "system", "content": "You are an AI research assistant acting as a critique"},
        {"role": "user", "content": f"Please critique the following summary:\n\n{summary}. Recognize flaws within the summary, offer constructive criticism, and pinpoint areas for improvement"}
    ]
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        max_tokens=500,
        stop=None,
        temperature=0.7,
    )
    critique = response.choices[0].message.content
    return critique

In [ ]:
def final_summary_based_on_critique(summary, critique):
    messages = [
        {"role": "system", "content": "You are an Ai assistant conducting research"},
        {"role": "user", "content": f"Enhance the provided summary: {summary} in accordance with the ensuing evaluation: {critique}."}
    ]
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        max_tokens=500,
        stop=None,
        temperature=0.7,
    )
    final_summary= response.choices[0].message.content
    return final_summary

In [ ]:
def research_agent():

  topic_of_research = input("Welcome! I am your research assistant, ready to help you discover relevant topics and provide valuable summaries. Please enter your research topic::\n\n\nTopic Of Research:")
  print("\n\n 1. Topic Breakdown Tool :")
  subqueries = topic_breakdown_tool(topic_of_research)
  print(f"\n {subqueries}")

  print("\n\n 2. Query Expansion Tool :")
  extended_subqueries = query_expansion_tool(subqueries)
  print(f"\n {extended_subqueries}")

  print("\n\n 3. Search Tool : ")
  search_results = search_tool(extended_subqueries)
  print(f"\n {search_results['hits']}")

  print("\n\n 4. Summary Tool :")
  summary = summary_tool(search_results['hits'])
  print(f"\n {summary}")

  print("\n\n 5. Critique Tool :")
  critic_results = critique_tool(summary)
  print(f"\n {critic_results}")

  print("\n\n 6. Final Summary Generator based on summary and crtique:")
  final_summary = final_summary_based_on_critique(summary, critic_results)
  print(f"\n {final_summary}")

In [ ]:
research_agent()

Welcome! I am your research assistant, ready to help you discover relevant topics and provide valuable summaries. Please enter your research topic::


Topic Of Research:Advancements in LLM


 1. Topic Breakdown Tool :

  Sure, I'd be happy to help generate some subqueries for the research topic "Advancements in Language Learning Models (LLM)". Here they are:

1. What are the latest architectural innovations in Language Learning Models, and how do they differ from previous models?
2. How have recent advancements in Natural Language Processing (NLP) techniques impacted the development of Language Learning Models?
3. What role do transfer learning and multitask learning play in the current advancements of Language Learning Models?
4. How do the ethical considerations of Language Learning Models, such as bias and fairness, impact their advancement and implementation?

These subqueries can help you explore different aspects of the overarching topic and provide a more comprehensive understan